## Using aeon distances with scikit learn

Scikit learn has a range of algorithms based on distances, including classifiers,
regressors and clusterers. These can (mostly) all be used with aeon distances in two
ways.

1. Pass the distance function as a callable to the `metric` parameter in
constructor or
2. Set the `metric` to precomputed in the constructor and pass a pairwise distance
matrix to `fit` and `predict`.

Please note that the SVM classifiers and regressors work differently:

1. If you pass a distance function as the `kernel`, then you also need to pass the
pairwise distance to `fit`. The callable , then you can only use the
estimator with univariate time series stored as 2D numpy.
2. Not all elastic distance functions are kernels. This included DTW.

In [7]:
# Classifiers and regressors: KNN and support vector machine
from sklearn.neighbors import KNeighborsClassifier  # KNeighborsRegressor
from sklearn.svm import SVC  # SVR

from aeon.datasets import load_unit_test
from aeon.distances import dtw_distance, dtw_pairwise_distance

trainx, trainy = load_unit_test(split="TRAIN", return_type="numpy2D")
testx, testy = load_unit_test(split="TEST", return_type="numpy2D")

# Use directly on TSC data with standard scikit functionality/distances etc
svm = SVC(kernel="poly")
knn = KNeighborsClassifier(metric="manhattan")
trainx, trainy = load_unit_test(return_type="numpy2D", split="train")
testx, testy = load_unit_test(return_type="numpy2D", split="test")
svm.fit(trainx, trainy)
print("SVM with polynomial kernel on 2D time series data = ", svm.predict(testx))
knn.fit(trainx, trainy)
print("KNN with manhatttan distance on 2D time series data = ", knn.predict(testx))

# Use case 1: KNN. pass a callable. 2D univariate equal length TSC only,
# Requires 2D numpy input of equal length series shape (n_cases, series_length)

knn = KNeighborsClassifier(metric=dtw_distance)
knn.fit(trainx, trainy)
print("KNN with on 2D time series data = ", knn.predict(testx))

# Use case 1: set precomputed, pass distances matrix to train and test.
# Can use any input compatible with pairwise_distance

trainx, trainy = load_unit_test(split="train")
testx, testy = load_unit_test(split="test")
matrix_train = dtw_pairwise_distance(trainx, trainx)
matrix_test = dtw_pairwise_distance(testx, trainx)

svm = SVC(kernel="precomputed")
knn = KNeighborsClassifier(metric="precomputed")
# svm.fit(matrix_test, trainy)
# print("SVM with ERP distance matrix = ", svm.predict(testx))
# knn.fit(matrix_test, trainy)
# print("KNN with ERP distance matrix = ", knn.predict(testx))

SVM with polynomial kernel on 2D time series data =  ['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '1' '2' '2' '1'
 '2' '2' '2' '2']
KNN with manhatttan distance on 2D time series data =  ['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '1' '2' '2' '1'
 '2' '2' '2' '1']
KNN with on 2D time series data =  ['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '1' '2' '2' '1'
 '2' '2' '2' '2']
